# LINKS DATASETS

https://www.kaggle.com/datasets/ashishjangra27/goodread-books-dataset (89k linhas)

https://www.kaggle.com/datasets/deepaktheanalyst/books-details-dataset (13k linhas)

https://www.kaggle.com/datasets/andkonar/book-dataset-300k-scraped-polish (352k linhas)

https://www.kaggle.com/datasets/daspias/best-books-dataset (52k linhas)

https://www.kaggle.com/datasets/arnabchaki/goodreads-best-books-ever (52k linhas)

https://www.kaggle.com/datasets/sp1thas/book-depository-dataset?select=dataset.csv (1.11m linhas)

https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews/data (212k linhas)  **ATUAL**







# IMPORTAÇÃO

In [1]:
# Importar as bibliotecas
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn.metrics
from scipy import stats
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [ ]:
# Ler o dataset
# df = pd.read_csv('G:\\Projeto-Integrador-VI\\Books_rating.csv')

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Informações de conexão com o banco de dados
db_info = {
    'host': 'node161687-env-8279702-review.jelastic.saveincloud.net',
    'user': 'root',
    'port': '14856',
    'password': 'NHBfah67653',
    'database': 'projeto_ia'
}

# Criação da conexão com o banco de dados usando SQLAlchemy
engine = create_engine(f"mysql+pymysql://{db_info['user']}:{db_info['password']}@{db_info['host']}:{db_info['port']}/{db_info['database']}")

# Aqui você pode substituir este exemplo por seus próprios dados ou lógica para criar um DataFrame
# df = ...

# Substituir o DataFrame diretamente para um banco de dados relacional
df.to_sql(name='reviews', con=engine, index=False, if_exists='replace')

df.to_parquet('output.parquet', index=False)


In [ ]:
print(df.head(50))

    product_id         user_id  \
0   1882931173   AVCGYZL8FQQTD   
1   0826414346  A30TK6U7DNS82R   
2   0826414346  A3UH4UZ4RSVO82   
3   0826414346  A2MVUWT453QH61   
4   0826414346  A22X4XUPKF66MR   
5   0826414346  A2F6NONFUDB6UK   
6   0826414346  A14OJS0VWMOSWO   
7   0826414346  A2RSSXTDZDUSH4   
8   0826414346  A25MD5I2GUIW6W   
9   0826414346  A3VA4XFS5WNJO3   
10  0829814000   AZ0IOBU20TBOP   
11  0829814000  A373VVEU6Z9M0N   
12  0829814000   AGKGOH65VTRR4   
13  0829814000   A3OQWLU31BU1Y   
14  0595344550  A3Q12RK71N74LB   
15  0595344550  A1E9M6APK30ZAU   
16  0595344550   AUR0VA5H0C66C   
17  0595344550  A1YLDZ3VHR6QPZ   
18  0595344550   ACO23CG8K8T77   
19  0595344550  A1VK81CRRC7MLM   
20  0595344550  A2GKUH6OBW7POH   
21  0595344550   A2BAF61XBWSLH   
22  0595344550   A6OQ83FD8E2OC   
23  0595344550   AVRN9E6X9I6KF   
24  0595344550  A2YWUISTJ728T5   
25  0595344550   ASEIU33YVKKWW   
26  0595344550   AJV5HX8BBZKEP   
27  0595344550   A1P2KGE2Z8RTA   
28  0595344550

# TRATAMENTO DOS DADOS

In [ ]:
# Selecionar colunas e renomear
df = df[['product_id', 'user_id', 'title', 'score', 'time']]
df.rename(columns={'Id':'product_id','User_id':'user_id','review/time':'time','Title':'title','review/score':'score'},inplace=True)
df.head()

df.head()

,product_id,user_id,title,score,time
0,1882931173,AVCGYZL8FQQTD,Its Only Art If Its Well Hung!,4.0,940636800
1,0826414346,A30TK6U7DNS82R,Dr. Seuss: American Icon,5.0,1095724800
2,0826414346,A3UH4UZ4RSVO82,Dr. Seuss: American Icon,5.0,1078790400
3,0826414346,A2MVUWT453QH61,Dr. Seuss: American Icon,4.0,1090713600
4,0826414346,A22X4XUPKF66MR,Dr. Seuss: American Icon,4.0,1107993600


In [ ]:
# Remover usuários com valores faltantes
df = df.dropna(subset=['user_id'])

In [ ]:
# Verificar tamanho das colunas product_id e user_id
print("Dimensão original")
print("Tamanho da coluna product_id:", len(df['product_id']))
print("Tamanho da coluna user_id:", len(df['user_id']))

# Definir valores limite
limite_product_id = 200
limite_user_id = 10

# Contar valores únicos das colunas product_id e user_id
cont_product_id = df['product_id'].value_counts()
cont_user_id = df['user_id'].value_counts()

# Filtrar as linhas abaixo do limite
df_filtrado = df[(df['product_id'].isin(cont_product_id[cont_product_id >= limite_product_id].index)) &
                 (df['user_id'].isin(cont_user_id[cont_user_id >= limite_user_id].index))]

print("\nDimensão filtrada")
print("Tamanho da coluna product_id:", len(df_filtrado['product_id']))
print("Tamanho da coluna user_id:", len(df_filtrado['user_id']))

Dimensão original
Tamanho da coluna product_id: 2438203
Tamanho da coluna user_id: 2438203

Dimensão filtrada
Tamanho da coluna product_id: 346920
Tamanho da coluna user_id: 346920


In [ ]:
# Extrair únicos de user_id e product_id
uni_user_ids = df_filtrado['user_id'].unique()
uni_product_ids = df_filtrado['product_id'].unique()

# Criar dicionários de user_id e product_id para índices
index_user_id = {user_id: index for index, user_id in enumerate(uni_user_ids)}
index_product_id = {product_id: index for index, product_id in enumerate(uni_product_ids)}


matriz = np.zeros((len(uni_user_ids), len(uni_product_ids)))

# Matriz tendo user_id como linhas, product_id como colunas com suas avaliações
for _, row in df_filtrado.iterrows():
    user_id = row['user_id']
    product_id = row['product_id']
    score = row['score']

    user_index = index_user_id[user_id]
    product_index = index_product_id[product_id]

    if matriz[user_index][product_index] < score:
        matriz[user_index][product_index] = score
print(matriz.shape)
matriz

(25751, 1572)


array([[5., 0., 0., ..., 0., 0., 0.],
       [5., 0., 0., ..., 0., 0., 0.],
       [5., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 5., 0., 0.],
       [0., 0., 0., ..., 0., 5., 0.],
       [0., 0., 0., ..., 0., 5., 0.]])

# Z-SCORE

In [ ]:
matriz = stats.zscore(matriz, axis=0)

# MSE, F1 SCORE, RECALL K

**Referências:**

https://grabngoinfo.com/the-ultimate-guide-to-evaluating-your-recommendation-system

https://towardsdatascience.com/evaluation-metrics-for-recommendation-systems-an-overview-71290690ecba

https://www.flai.com.br/juscudilio/qual-a-melhor-metrica-para-avaliar-os-modelos-de-machine-learning

https://www.shaped.ai/blog/evaluating-recommendation-systems-part-1

In [ ]:
def calcular_mse(matriz_previsao, matriz_teste):
    """
    Calcular o erro quadrático médio (MSE) entre as previsões e os valores reais

    Parâmetros:
        matriz_previsao: matriz de previsões do sistema de recomendação
        matriz_teste: matriz de valores reais

    Retorna:
        Erro quadrático médio (MSE)
    """
    return sklearn.metrics.mean_squared_error(matriz_teste, matriz_previsao)


def calcular_f1_score(recall, precisao):
    """
    Calcular o F1-score entre a precisão e o recall

    Parâmetros:
        recall: precisão das previsões
        precisao: recall das previsões

    Retorna:
        F1-score
    """

     # Calcular o F1-score entre a precisão e o recall
    if recall + precisao == 0:
        return 0
    f1_score = 2 * (precisao * recall) / (precisao + recall)
    return f1_score


def precisao_k(matriz_reais, matriz_previsao, k, limite):
    """
    Calcular a precisão das previsões, considerando apenas os k itens com maior pontuação

    Parâmetros:
        matriz_reais: matriz de valores reais
        matriz_previsao: matriz de previsões do sistema de recomendação
        k: número de itens considerados.
        limite: limite de pontuação para considerar um item como relevante

    Retorna:
        Precisão das previsões
    """

    # Converter as previsões para valores binários
    matriz_previsoes_binarias = matriz_previsao >= limite

    # Calcular a precisão das previsões, considerando apenas os k itens com maior pontuação
    precisao = []
    for i in range(len(matriz_reais)):
        indices_reais = np.where(matriz_reais[i] >= limite)[0]
        indices_previsoes = np.argsort(~matriz_previsoes_binarias[i])[:k]
        indices_comuns = np.intersect1d(indices_reais, indices_previsoes)
        precisao.append(len(indices_comuns) / len(indices_previsoes))

    return np.mean(precisao)


def recall_k(matriz_verdadeira, matriz_preditiva, k, limite):
    """
    Calcular o recall das previsões, considerando apenas os k itens com maior pontuação

    Parâmetros:
        matriz_verdadeira: matriz de valores reais.
        matriz_preditiva: matriz de previsões do sistema de recomendação.
        k: número de itens considerados.
        limite: limite de pontuação para considerar um item como relevante.

    Retorna:
        Recall das previsões.
    """

     # Ordenar as previsões do sistema de recomendação em ordem decrescente
    matriz_predita_ordenada = np.argsort(matriz_preditiva, axis=1)[:, ::-1][:, :k]

    # Calcular o recall das previsões, considerando apenas os k itens com maior pontuação
    scores_recall = []
    for i in range(len(matriz_verdadeira)):
        verdadeiros_positivos = len(set(matriz_predita_ordenada[i]).intersection(set(np.where(matriz_verdadeira[i] >= limite)[0])))
        positivos_reais = len(np.where(matriz_verdadeira[i] >= limite)[0])
        if positivos_reais > 0:
            scores_recall.append(verdadeiros_positivos / positivos_reais)
    recall = np.mean(scores_recall)
    return recall

# SVD

**Referências:**

https://analyticsindiamag.com/singular-value-decomposition-svd-application-recommender-system

https://jaketae.github.io/study/svd

https://www.freecodecamp.org/news/singular-value-decomposition-vs-matrix-factorization-in-recommender-systems-b1e99bc73599

https://medium.com/data-hackers/criando-sistemas-de-recomenda%C3%A7%C3%A3o-em-python-ef350f601e3d

In [ ]:
def dividir_treino_teste(matriz, test_size=0.2, random_state=42):
    matriz_treino, matriz_teste = train_test_split(matriz, test_size=test_size, random_state=random_state)
    return matriz_treino, matriz_teste

In [ ]:
def calcular_svd(matriz_treino, k=897):
    """
    Calcular a decomposição em valores singulares (SVD) da matriz de treinamento

    Parâmetros:
        matriz_treino: a matriz de treinamento.
        k: o número de componentes principais a serem usados (padrão é 600)

    Retorna:
        Uma tupla contendo as matrizes U, S e V
    """

    # Converter a matriz para um formato esparsa para melhorar o desempenho
    matriz_treino_sparse = csr_matrix(matriz_treino)
    # Calcular a decomposição SVD da matriz de treinamento
    U_treino, S_treino, VT_treino = svds(matriz_treino_sparse, k=k)
    # Reordenar as matrizes U e V para que os componentes principais estejam na ordem decrescente de importância
    S_treino_k = np.diag(S_treino[::-1])
    U_treino_k = U_treino[:, ::-1]
    VT_treino_k  = VT_treino[::-1, :]

    return U_treino_k, S_treino_k, VT_treino_k

In [ ]:
# Dividir os dados em conjuntos de treinamento e teste
matriz_treino, matriz_teste = dividir_treino_teste(matriz)

# Calcular a SVD da matriz de treinamento
U_treino, S_treino, VT_treino = calcular_svd(matriz_treino)

# Realizar previsões para o conjunto de treinamento
U_treino_pred = np.dot(matriz_treino, VT_treino.T)
matriz_previsao_treino  = np.dot(U_treino_pred, VT_treino)

# Realizar previsões para o conjunto de teste
U_teste_pred = np.dot(matriz_teste, VT_treino.T)
matriz_previsao = np.dot(U_teste_pred, VT_treino)

mse_treino = calcular_mse(matriz_treino, matriz_previsao_treino )
mse_teste = calcular_mse(matriz_teste, matriz_previsao)

print("Erro Quadrático Médio do Conjunto de Treino (MSE):", mse_treino)
print("Erro Quadrático Médio do Conjunto de Teste (MSE):", mse_teste)

Erro Quadrático Médio do Conjunto de Treino (MSE): 0.00010855873828068341
Erro Quadrático Médio do Conjunto de Teste (MSE): 0.00048623316365041075


In [ ]:
  # Calcular a precisão e recall para o conjunto de teste com os 10 principais itens recomendados
  precisao = precisao_k(matriz_teste, matriz_previsao, k=10, limite=3)

  # Calcular o F1 Score com base nos valores de precisão e recall
  recall = recall_k(matriz_teste, matriz_previsao, k=10, limite=3)

  f1_score = calcular_f1_score(recall, precisao)
  print("MSE de treinamento:", np.sqrt(mse_treino) )
  print("MSE de teste:", np.sqrt(mse_teste))
  print("Precisão:", precisao)
  print("Recall:", recall)
  print("F1 Score:", f1_score)

MSE de treinamento: 0.01041915247420266
MSE de teste: 0.022050695309908275
Precisão: 0.7162104445738691
Recall: 0.8335255575845498
F1 Score: 0.7704276203558023


# SAÍDA

In [ ]:
print(index_user_id.keys())

dict_keys(['AF3X7J0XC391L', 'A3FHSO1SKHU378', 'A3OWUSU9RG4NMF', 'ATMOVG6SV5D2B', 'A16QJ649N8PRV', 'A3QQLPR8R87ONM', 'A321W4SSC0F6AP', 'AYJ9UC3A6JZYJ', 'ADJXL0W0RJ7VV', 'A2NVQM5H3RS1SL', 'A3JPFWKS83R49V', 'A2TWZX3FBC7WK7', 'A3EN6NDS6S7N9N', 'A25PAECTYS2CDO', 'A1NPY3BZBWGD9X', 'A2441CXD8TXS0R', 'A38IZVTPUJEP40', 'A2NU4WC77YNA8H', 'A213YKLZ250CQ6', 'A2FTHCGH06O4Y5', 'A1KFCZF2V2FS62', 'A18I8EYPJK3XMA', 'ATSRE85IKZJ3Z', 'AW297AHRU4Y4M', 'A1VA170GUYGTB', 'A1UI9T8WKJPZN5', 'ATSUQCIFVFDTI', 'A12M1WWAGSH35S', 'A1LIYCSF2MDWFB', 'A2XF9FRO6OWZCH', 'A2XEVM311YJJGF', 'A3MVU8X8EC9VRT', 'A2GUHY6OBKZESB', 'A1NJ7YZYSVZC0', 'AZQ3ERH4TR6N0', 'ADIB6IP2IWMT4', 'A336FCS64Y5JYX', 'A31PI1S4YUTNFB', 'AVW8VQ5YFK09F', 'A31D2B0N48J4H7', 'A8SWO9OJWHDSM', 'A19QW5QUH2APMK', 'A21M9OUEY2B01X', 'A2K5VQV3F7HPBT', 'A2TIJREKDLV2S1', 'AHY8ZUJ8NMKT9', 'A2AJ4QP5SOCJZ2', 'A2MI5PQBEII7O3', 'A37IH65E7CTXRS', 'A1DKMAQDFN0UWA', 'A3EFCK0HZQPNQI', 'AMGM6T05KDZI5', 'A1I78HZLE3O1SD', 'AN7F7HY1T6AAN', 'A1L1ZYOS0YX21P', 'A9NFFAZF3LIB2',

In [ ]:
user_id = "AVCGYZL8FQQTD"

# Verificar se o usuário existe
if user_id in index_user_id.keys():
    print(f"Usuário {user_id} existe.")
else:
    print(f"Usuário {user_id} não existe.")

Usuário AVCGYZL8FQQTD existe.


In [ ]:
def buscar_itens_relevantes_para_usuario(user_id, itens_relevantes=5):
    # Obter o índice do usuário
    indice_usuario = index_user_id[user_id]
    embedding_usuario = U_treino[indice_usuario, :]

    pontuacoes_similaridade = VT_treino.T.dot(embedding_usuario)

    indices_ordenados = pontuacoes_similaridade.argsort()[::-1]
    top_indices_relevantes = indices_ordenados[:itens_relevantes]

    itens_relevantes = [list(index_product_id.keys())[list(index_product_id.values()).index(idx)] for idx in top_indices_relevantes]
    titulos_relevantes = df.loc[df['product_id'].isin(itens_relevantes), 'title'].tolist()

    # Remover quaisquer títulos duplicados
    titulos_unicos_relevantes = list(set(titulos_relevantes))

    # Obter o conjunto final de itens relevantes sem títulos duplicados
    itens_relevantes_finais = []
    for titulo in titulos_unicos_relevantes:
        itens_relevantes_finais.append(titulo)

    return itens_relevantes_finais

user_id = ["AVCGYZL8FQQTD", "A3FHSO1SKHU378", "A3OWUSU9RG4NMF"]
top_n = 5

for id in user_id:
    itens_relevantes = buscar_itens_relevantes_para_usuario(id, top_n)
    print(f"Usuário: {id}")
    print("Itens Relevantes:")
    for i, item in enumerate(itens_relevantes):
        print(f"{i+1}. {item}")
    print()


    

Usuário: AF3X7J0XC391L
Itens Relevantes:
1. Mansfield Park (Coles Notes)
2. John Adams
3. Mansfield Park
4. Good to Great
5. Mansfield Park (A Watts Ultratype Edition)

Usuário: A3FHSO1SKHU378
Itens Relevantes:
1. Stranger in a Strange Land
2. Stanger in a Strange Land
3. Illusions: The Adventures of a Reluctant Messiah
4. Bridges of Madison County

Usuário: A3OWUSU9RG4NMF
Itens Relevantes:
1. America Alone: The End of the World as We Know It
2. Gnostic Gospels
3. Maui Revealed: The Ultimate Guidebook
4. Heart of Darkness
5. The Case for Faith: A Journalist Investigates the Toughest Objections to Christianity

